**Numerical Methods Coursework 1**

In [ ]:
import numpy as np
import timeit

Firstly we define the Kamrad-Ritchken model.

In [ ]:

def create_trinomial_model_kamrad_ritchken(s0, r, dt, sigma, N, lamda):
    #model parameters
    
    u=np.exp(lamda*sigma*np.sqrt(dt))
    qu = 1/(2*lamda**2) + ((r - sigma**2/2)*np.sqrt(dt))/2*lamda*sigma
    qm = 1-1/lamda**2
    qd = 1/(2*lamda**2) - ((r - sigma**2/2)*np.sqrt(dt))/2*lamda*sigma

    if (qu < 0 or qd < 0):
        print("The value of dt is too large and the probabilities will not be viable, please use a smaller value for dt.")
        return

    # Set up the tree via S[n,k] = s0 * u^(2k-n)
    prices = np.zeros((N+1, 2*N+1))
    for t in range(N+1):
        prices[t, 0:2*t+1] = s0 * u**np.arange(-t, t+1, 1)

    return (prices, np.array([qd, qm, qu]), u)

    
        

We create an array of prices for $A^n_j$ noticing that these prices have no dependence on $n$.

In [ ]:
def create_auxillary_values(s0, N, u):
    indices = np.arange(0,N+1)
    AValues = s0*u**indices
    return AValues

Next we define the funtion for generating the prices by backward induction

In [ ]:
def generate_lookback_prices(s0, r, dt, sigma, N, lamda, american = False):
    Nfloor_div_2 = int(np.floor(N/2.))     
    prices, probabilities, u = create_trinomial_model_kamrad_ritchken(s0, r, dt, sigma,  N, lamda)
    auxillary_values = create_auxillary_values(s0, N, u)
            
    # Option values matrix indexed by time, k, j
    option_values = np.zeros([N+1, 2*N+1, Nfloor_div_2+1])

    # Terminal condition: option values given by terminal payoff

    terminal_stock_price = prices[N]
    k_range = np.arange(0,2*N + 1)
    j_range = np.arange(0, Nfloor_div_2+1)
    auxiliary_variable_index = np.floor(k_range/2)[:,np.newaxis] = Nfloor_div_2 + j_range[np.newaxis, :]
    option_values[N] = np.take_along_axis(auxillary_values[:,np.newaxis], auxiliary_variable_index, axis = 0) - terminal_stock_price[:,np.newaxis]
    print(option_values[N])
            
    # Backward induction

    # Loop in time
    for n in reversed(range(N)):
               
        #generate the possible values of k_new indexed by current k, stock movement where for the stock movement 0 is down, 1 is no movement, 2 is up.
        k_new = np.column_stack([np.arange(0, 2*n+1), np.arange(1,2*n+2), np.arange(2,2*n+3)])

        j_range = np.arange(0,n+1)

        #Use the shooting function to generate the possible values of jnew indexed by current k, current j, stock movement where for the stock movement 0 is down, 1 is no movement, 2 is up.
        phi_result = np.maximum(j_range[np.newaxis,:,np.newaxis], k_new[:,np.newaxis,:] -n -1)

        #calculate the option prices using backward induction
        
        #restrict the option_values to just legal values of k at time n+1
        option_values_restricted = option_values[n+1, 0:phi_result.shape[0], :]

        current_time_values = (probabilities * np.take_along_axis(option_values_restricted[:,:,np.newaxis], phi_result, axis = 1)).sum(axis=2)
        
        option_values[n][0:2*n+1,0:n+1] =  np.exp(-1*r*dt)*current_time_values
    
    return option_values

*Finally let's run the code to obtain the price

In [ ]:
s0 = 100
r = 0
dt = 1
sigma = 0.1
N = 1
lamda = 1

generate_lookback_prices(s0, r, dt, sigma, N, lamda, american=False)